In [1]:
import pandas as pd
import numpy as np
import os

os.listdir('../input')

['df_ref2_session.csv',
 'new_submission_dwell_time.csv',
 'new_submission_baseline.csv',
 'df_ref.csv',
 'df_dwell_time_session.csv',
 'ref_count2_baselineOnly_pred.csv',
 'train.csv',
 'df_ref2.csv',
 'submission_popular.csv',
 'item_metadata.csv',
 'df_full_except_noinfo.csv',
 'test.csv',
 'new_submission.csv',
 'dwell_df.csv',
 'baseline_sorting.csv',
 'dwell_time_agg.csv',
 'test2.csv']

In [2]:
path = '../input/'

In [3]:
df = pd.read_csv(path + 'df_full_except_noinfo.csv')

In [4]:
def only_number(string):
    """reference에서 숫자인 것만 뽑아 int type으로 변환"""
    try:
        int(string)
        return string
    except ValueError:
        return np.nan
    
df.reference = df.reference.apply(only_number)

In [5]:
df.reset_index(drop = True, inplace = True)

In [6]:
ref_list = list(df.reference.values)
first_index = [i for i, r in enumerate(ref_list) if ref_list[i-1] != ref_list[i]]

In [7]:
dwell_df = pd.DataFrame(columns = ['user_id','session_id','reference','dwell_time'])
dwell_df['user_id'] = df.loc[first_index, 'user_id']
dwell_df['session_id'] = df.loc[first_index, 'session_id']
dwell_df['reference'] = df.loc[first_index, 'reference']
dwell_df.reset_index(drop = True, inplace = True)

first_time = df.loc[first_index, 'timestamp']
first_time = pd.to_datetime(first_time, unit = 's')
first_time.reset_index(drop = True, inplace = True)

last_time = pd.Series(np.roll(first_time,-1))

dwell_df['dwell_time'] = (last_time - first_time).astype('timedelta64[s]')

In [8]:
dwell_df.head()

,user_id,session_id,reference,dwell_time
0,00RL8Z82B2Z1,aff3928535f48,NaN,62.0
1,00RL8Z82B2Z1,aff3928535f48,666856,10.0
2,00RL8Z82B2Z1,aff3928535f48,109038,0.0
3,00RL8Z82B2Z1,aff3928535f48,666856,0.0
4,00RL8Z82B2Z1,aff3928535f48,109038,0.0


In [9]:
dwell_df.dropna(inplace = True)
dwell_df = dwell_df[dwell_df.dwell_time>0]
dwell_df.shape

(2249506, 4)

In [10]:
dwell_df.dwell_time = dwell_df.dwell_time.apply(int)

In [14]:
dwell_df = dwell_df.groupby(['session_id', 'reference'])['dwell_time'].sum().to_frame().reset_index()
dwell_df.head(3)

,session_id,reference,dwell_time
0,000056cd97ce2,2784325,28
1,000056cd97ce2,3811810,23559
2,00008c1f39c46,3501822,41422


In [23]:
dwell_df = dwell_df.groupby('session_id').apply(lambda x : x.sort_values(['dwell_time'], ascending = False)).reset_index(drop = True)

In [26]:
#dwell_df.to_csv(path + 'df_dwell_time_session.csv', index = False)

# sort by dwell time

In [27]:
new_submission = pd.read_csv(path + 'new_submission.csv')
new_submission['impressions'] = new_submission['impressions'].apply(lambda x : x.split('|'))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ..."


In [29]:
dwell_time_sort = dwell_df.groupby('session_id')['reference'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
dwell_time_sort = dwell_time_sort[dwell_time_sort.session_id.isin(new_submission.session_id)]
dwell_time_sort.head()

,session_id,reference
6,0000e0500c2e7,9387530 3844380
10,000177b850519,17011 17266
11,00017b3b2c136,6698142 6298194
24,000246890bfc6,386136 98916 98934 98938 98935
35,000338c4794e8,5574632


In [30]:
dwell_time_sort.reference = dwell_time_sort.reference.apply(lambda x : x.split(' '))
dwell_time_sort.rename(columns = {'reference' : 'dwell_time_sort_sorted'}, inplace = True)
dwell_time_sort.head(3)

,session_id,dwell_time_sort_sorted
6,0000e0500c2e7,"[9387530, 3844380]"
10,000177b850519,"[17011, 17266]"
11,00017b3b2c136,"[6698142, 6298194]"


In [31]:
new_submission = pd.merge(new_submission, dwell_time_sort, on = ['session_id'], how = 'left')

In [32]:
def sort_dwell(imp, ref) :
    my = []
    for rr in ref :
        if rr in imp :
            my.append(rr)
            imp.remove(rr)
    imp = my + imp
    return imp

In [33]:
#apply 
apply_index = new_submission[new_submission.session_id.isin(dwell_time_sort.session_id)].index
new_submission.loc[apply_index, 'impressions'] = new_submission.loc[apply_index].apply(lambda x : sort_dwell(x['impressions'], x['dwell_time_sort_sorted']), axis = 1)

In [34]:
def mrr(answer, imp) :
    myscore = 0
    
    for i,j in zip(answer, imp) :
        if i in j :
            myscore += 1/(j.index(i) + 1)
    print(myscore / len(imp))

In [35]:
new_submission.reference = new_submission.reference.apply(str)
mrr(new_submission['reference'], new_submission['impressions'])

0.7136369290876735


In [37]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[3241426, 1050068, 2059240, 2033381, 1724779, ...","[3241426, 1050068, 2059240]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590]


In [38]:
new_submission.impressions = new_submission.impressions.apply(lambda x : ' '.join(x))
ind = new_submission.loc[~new_submission.dwell_time_sort_sorted.isnull()].index
new_submission.loc[ind, 'dwell_time_sort_sorted'] = new_submission.loc[ind]['dwell_time_sort_sorted'].apply(lambda x : ' '.join(x))
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,3241426 1050068 2059240 2033381 1724779 127131...,3241426 1050068 2059240
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [39]:
#new_submission.to_csv(path + 'new_submission_dwell_time.csv', index = False)

# baseline(ref count) + dwell_time

In [40]:
dwell_sorted = pd.read_csv(path + 'new_submission_dwell_time.csv')
dwell_sorted.drop('impressions', axis = 1, inplace = True)
idx = ~dwell_sorted.dwell_time_sort_sorted.isnull()
dwell_sorted.loc[idx, 'dwell_time_sort_sorted'] = dwell_sorted.loc[idx, 'dwell_time_sort_sorted'].apply(lambda x : x.split(' '))
dwell_sorted.head(3)

,user_id,session_id,timestamp,step,reference,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[3241426, 1050068, 2059240]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,[2817590]


In [42]:
baseline = pd.read_csv(path + 'new_submission_baseline.csv')
idx = ~baseline.ref_count_sorted.isnull()
baseline.impressions = baseline.impressions.apply(lambda x : x.split(' '))
baseline.loc[idx, 'ref_count_sorted'] = baseline.loc[idx, 'ref_count_sorted'].apply(lambda x : x.split(' '))
baseline.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[1050068, 2059240, 3241426, 2033381, 1724779, ...","[1050068, 2059240, 3241426]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590]


In [43]:
print('dwell sort shape : ', dwell_sorted.shape)
print('baseline shape : ', baseline.shape)

dwell sort shape :  (121526, 6)
baseline shape :  (121526, 7)


In [44]:
baseline = pd.merge(baseline, dwell_sorted)
baseline.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted,dwell_time_sort_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[1050068, 2059240, 3241426, 2033381, 1724779, ...","[1050068, 2059240, 3241426]","[3241426, 1050068, 2059240]"
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590],[2817590]


In [45]:
def sort_again(imp, ref_sorted, iid) :
    
    if type(ref_sorted)!= list :
        return imp
    
    if type(iid)!= list :
        return imp
    

    i = 0
    for ref in ref_sorted :
        if ref in imp :
            i+=1
            if ref in iid :
                j = iid.index(ref)
                iid = iid[:j] + iid[j+1:]


    final_list = imp[:i] + iid + list(set(imp[i:]) - set(iid))
    return final_list

In [46]:
baseline['sort_again'] = baseline.apply(lambda x : sort_again(x['impressions'], x['ref_count_sorted'], x['dwell_time_sort_sorted']), axis = 1)

In [47]:
baseline.reference = baseline.reference.apply(str)
mrr(baseline.reference, baseline.sort_again)

0.7671447170738503
